In [1]:
#all our necessary libraries

import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tqdm import tqdm
from PIL import Image
import time
import pygame
from IPython.display import display, clear_output

pygame 2.5.2 (SDL 2.28.3, Python 3.8.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#training and test samples

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
#output of the first 25 numbers using matplotlib
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(train_images[i], cmap='gray')
    plt.title(f"Label: {train_labels[i]}")
    plt.axis('off')
plt.show()

In [ ]:
#function conversion to probabilities
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

#it is a classic of learning, but I need mainly its derivative from it to adjust the weights and not only
def sigmoid(x, deriv = False):
    if deriv == True:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

def relu(x): 
    return np.maximum(0.0, x)

#here my personal functions begin, the first is to output which numbers are in the test sample
def answer(pred):
    output = []
    for i in range(len(pred)):
        pred_list = list(pred[i])
        output.append(pred_list.index(max(pred[i])))
    return output

#accuracy of the neural network response
def percent_true_answer(output, target):
    outputs = 0
    for i in range(len(output)):
        pred_list = list(output[i])
        if pred_list.index(max(output[i])) == target[i]:
            outputs += 1

    return outputs

In [ ]:
#normalization of input training data
matrix_x = train_images


num_samples, num_rows, num_columns = matrix_x.shape

#сonverting the array to a two-dimensional format to apply normalization to each column
matrix_x_reshaped = matrix_x.reshape(num_samples, -1)

#сreating an instance of MinMaxScaler
scaler = preprocessing.MinMaxScaler()

#we normalize with respect to each column
scaler.fit(matrix_x_reshaped)
matrix_x_normalized_reshaped = scaler.transform(matrix_x_reshaped)

#returning the array to its original form
matrix_x = matrix_x_normalized_reshaped.reshape(num_samples, num_rows, num_columns)

#assigning the matrtix_target variable to the train_labels variable will be our test Y
matrix_target = train_labels

In [ ]:
#normalization of input test data 

matrix_test_x = test_images

num_samples_test, num_rows_test, num_columns_test = matrix_test_x.shape

matrix_test_x_reshaped = matrix_test_x.reshape(num_samples_test, -1)

scaler.fit(matrix_test_x_reshaped)
matrix_test_x_normalized_reshaped = scaler.transform(matrix_test_x_reshaped)

matrix_test_x = matrix_test_x_normalized_reshaped.reshape(num_samples_test, num_rows_test, num_columns_test) 

matrix_test_target = test_labels

In [ ]:
#the One Hot Encoding method for representing any numbers as zeros and one unit

encoding = preprocessing.OneHotEncoder(sparse = False)
matrix_target = encoding.fit_transform(matrix_target.reshape(-1, 1))
matrix_test_target = encoding.fit_transform(matrix_test_target.reshape(-1, 1))

In [ ]:
#initialization of weigths

INP_DIM = len(matrix_x_reshaped.T)
H_DIM = 350
OUT_DIM = len(matrix_target.T)

weights_1 = np.random.random((INP_DIM, H_DIM))
weights_2 = np.random.random((H_DIM, OUT_DIM))

In [ ]:
#learnng network

for _ in tqdm(range(1000)):
    #forward popagation
    z_1 = np.dot(matrix_x_normalized_reshaped, weights_1)
    a_1 = relu(preprocessing.normalize(z_1))

    z_2 = np.dot(a_1, weights_2)
    output = softmax(z_2)
    
    #back popagation
    error_2 = matrix_target - output
    deriv_2 = error_2 * sigmoid(output, deriv = True)

    error_1 = np.dot(deriv_2, weights_2.T)
    deriv_1 = error_1 * sigmoid(a_1, deriv = True)

    weights_2 += np.dot(a_1.T, deriv_2) * 0.01
    weights_1 += np.dot(matrix_x_normalized_reshaped.T, deriv_1) * 0.01
    

In [26]:
#use it if you want to see the accuracy on a test sample and you know the weights

z_1 = np.dot(matrix_x_normalized_reshaped, weights_1)
a_1 = relu(preprocessing.normalize(z_1))

z_2 = np.dot(a_1, weights_2)
output = softmax(z_2)

In [28]:
percent_true_answer(output, train_labels) / len(train_labels)

0.9523

In [ ]:
#reading the already configured weights

weights_1 = np.array(pd.read_csv("weights_1.csv"))
weights_2 = np.array(pd.read_csv("weights_2.csv"))


In [ ]:
#a drawing for determining numbers

pygame.init()

#window and canvas dimensions
WIDTH, HEIGHT = 400, 400
CANVAS_SIZE = 28
CELL_SIZE = WIDTH // CANVAS_SIZE

#colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

#create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Рисование")

#create мmatrix for storage information about
matrix = np.zeros((CANVAS_SIZE, CANVAS_SIZE), dtype=int)

drawing = False

#flag for loop control 
running = True

last_print_time = time.time()

#main loop
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            drawing = True
        elif event.type == pygame.MOUSEBUTTONUP and event.button == 1:
            drawing = False
        elif event.type == pygame.MOUSEMOTION and drawing:
            #we get the mouse coordinates and convert them to the coordinates of the cell on the canvas
            
            x, y = event.pos
            x //= CELL_SIZE
            y //= CELL_SIZE
            
            #setting the value 255 (white) in the central cell
            matrix[y][x] = 255
                                
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_c:
            #if the 'c' key is pressed, we clear the canvas(fill the matrix with zeroes)
            matrix.fill(0)

    #renderin on the screen
    screen.fill(BLACK)

    #we go through each cell of the matrix
    for y in range(CANVAS_SIZE):
        for x in range(CANVAS_SIZE):
            if matrix[y][x] == 255:  #if the cell value is 255 (the point you are drawing)
                #draw a white rectangle at the position of this cell
                if matrix[y][x] == 255:
                    pygame.draw.rect(screen, WHITE, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                
    
                #draw 4 more gray squares around this cell
                for dy, dx in [(0, -1), (-1, 0), (0, 1), (1, 0)]:
                    #we check that the neighboring cell is located within the canvas
                    if 0 <= y + dy < CANVAS_SIZE and 0 <= x + dx < CANVAS_SIZE:
                        #check if there is no white color in the next cell
                        if matrix[y + dy][x + dx] != 255:
                            #draw a gray rectangle in the next cell
                            pygame.draw.rect(screen, (100, 100, 100), ((x + dx) * CELL_SIZE, (y + dy) * CELL_SIZE, CELL_SIZE, CELL_SIZE))



    scaler.fit(matrix)

    dinamic_data = scaler.transform(matrix)
    dinamic_data = dinamic_data.reshape(28**2)

    dinamic_z_1 = np.dot(np.array([dinamic_data]), weights_1)
    dinamic_z_2 = np.dot(dinamic_z_1, weights_2)

    dinamic_out = softmax(dinamic_z_2)

    pygame.display.flip()

    if time.time() - last_print_time >= 0.01:
        clear_output(wait = True)
        display(f"This is number - {int(*answer(dinamic_out))} || Neuron number [{int(*answer(dinamic_out)) + 1}] || activation - {max(*dinamic_out)}")
        last_print_time = time.time()

#finish Pygame
pygame.quit()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [ ]:
#output of the first 25 numbers of the test sample
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(test_images[i], cmap='gray')
    plt.title(f"Label: {test_labels[i]}")
    plt.axis('off')
plt.show()

In [ ]:
#substituting weights for a test sample
z_1_test = np.dot(matrix_test_x_normalized_reshaped, weights_1)
a_1_test = relu(z_1_test)
z_2_test = np.dot(a_1_test, weights_2)
output_test = softmax(z_2_test)
error_test = matrix_test_target - output_test

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

answer(output_test[:25])

In [29]:
#the output of the accuracy of the neural network on the test sample
percent_true_answer(output_test, test_labels) / len(matrix_test_x)

0.9504

In [ ]:
#save weigths in csv files

weights_1_df = pd.DataFrame(weights_1)
weights_2_df = pd.DataFrame(weights_2)

weights_1_df.to_csv("weights_1.csv", index = False)
weights_2_df.to_csv("weights_2.csv", index = False)